# Rf 2 0 1_Composite
Addition and convolution: composite p.d.f with signal and background component

 pdf = f_bkg * bkg(x,a0,a1) + (1-fbkg) * (f_sig1 * sig1(x,m,s1 + (1-f_sig1) * sig2(x,m,s2)))



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:42 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Setup component pdfs
 ---------------------------------------

Declare observable x

In [3]:
RooRealVar x("x", "x", 0, 10);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Create two gaussian pdfs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [4]:
RooRealVar mean("mean", "mean of gaussians", 5);
RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
RooRealVar sigma2("sigma2", "width of gaussians", 1);

RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

Build chebychev polynomial p.d.f.

In [5]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0., 1.);
RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

---------------------------------------------
METHOD 1-Two RooAddPdfs
 =============================================

Add signal components
 ------------------------------------------

Sum the signal components into a composite signal p.d.f.

In [6]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

Add signal and background
 ------------------------------------------------

Sum the composite signal and background

In [7]:
RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

Sample, fit and plot model
 ---------------------------------------------------

Generate a data sample of 1000 events in x from model

In [8]:
RooDataSet *data = model.generate(x, 1000);

Fit model to data

In [9]:
model.fitTo(*data);

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions have been identified as constant and will be precalculated and cached: (sig1,sig2)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (bkg)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     3 bkgfrac      5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     4 sig1frac     8.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR           1
 **********
 NOW USING STRATEGY  1: TRY TO BALANCE SP

Plot data and pdf overlaid

In [10]:
RooPlot *xframe = x.frame(Title("Example of composite pdf=(sig1+sig2)+bkg"));
data->plotOn(xframe);
model.plotOn(xframe);

Overlay the background component of model with a dashed line

In [11]:
model.plotOn(xframe, Components(bkg), LineStyle(kDashed));

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()


Overlay the background+sig2 components of model with a dotted line

In [12]:
model.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineStyle(kDotted));

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg,sig2)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: (sig)


Print structure of composite p.d.f.

In [13]:
model.Print("t");

0x7fdf49b2e3a8 RooAddPdf::model = 0.898624 [Auto,Dirty] 
  0x7fdf4add8b08/V- RooChebychev::bkg = 0.79893 [Auto,Dirty] 
    0x7fdf4add8010/V- RooRealVar::x = 5
    0x7fdf4add83b8/V- RooRealVar::a0 = 0.441621 +/- 0.0731875
    0x7fdf4add8760/V- RooRealVar::a1 = 0.20107 +/- 0.117637
  0x7fdf49b2e000/V- RooRealVar::bkgfrac = 0.504184 +/- 0.0359091
  0x7fdf49b36870/V- RooAddPdf::sig = 1 [Auto,Dirty] 
    0x7fdf49b35af8/V- RooGaussian::sig1 = 1 [Auto,Dirty] 
      0x7fdf4add8010/V- RooRealVar::x = 5
      0x7fdf49b35000/V- RooRealVar::mean = 5
      0x7fdf49b353a8/V- RooRealVar::sigma1 = 0.5
    0x7fdf49b364c8/V- RooRealVar::sig1frac = 0.837334 +/- 0.116852
    0x7fdf49b35fe0/V- RooGaussian::sig2 = 1 [Auto,Dirty] 
      0x7fdf4add8010/V- RooRealVar::x = 5
      0x7fdf49b35000/V- RooRealVar::mean = 5
      0x7fdf49b35750/V- RooRealVar::sigma2 = 1


---------------------------------------------------------------------------------------------
METHOD 2-One RooAddPdf with recursive fractions
 =============================================================================================

Construct sum of models on one go using recursive fraction interpretations

   model2 = bkg + (sig1 + sig2)

In [14]:
RooAddPdf model2("model", "g1+g2+a", RooArgList(bkg, sig1, sig2), RooArgList(bkgfrac, sig1frac), kTRUE);

Nb: each coefficient is interpreted as the fraction of the
 left-hand component of the i-th recursive sum, i.e.

   sum4 = A + ( B + ( C + D)  with fraction fA, fB and fC expands to

   sum4 = fA*A + (1-fA)*(fB*B + (1-fB)*(fC*C + (1-fC)*D))

Plot recursive addition model
 ---------------------------------------------------------

In [15]:
model2.plotOn(xframe, LineColor(kRed), LineStyle(kDashed));
model2.plotOn(xframe, Components(RooArgSet(bkg, sig2)), LineColor(kRed), LineStyle(kDashed));
model2.Print("t");

[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (bkg,sig2)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
0x7fdf3d22f000 RooAddPdf::model = 0.898624 [Auto,Dirty] 
  0x7fdf4add8b08/V- RooChebychev::bkg = 0.79893 [Auto,Dirty] 
    0x7fdf4add8010/V- RooRealVar::x = 5
    0x7fdf4add83b8/V- RooRealVar::a0 = 0.441621 +/- 0.0731875
    0x7fdf4add8760/V- RooRealVar::a1 = 0.20107 +/- 0.117637
  0x7fdf49b2e000/V- RooRealVar::bkgfrac = 0.504184 +/- 0.0359091
  0x7fdf49b35af8/V- RooGaussian::sig1 = 1 [Auto,Dirty] 
    0x7fdf4add8010/V- RooRealVar::x = 5
    0x7fdf49b35000/V- RooRealVar::mean = 5
    0x7fdf49b353a8/V- RooRealVar::sigma1 = 0.5
  0x7fdf28041b00/V- RooRecursiveFraction::model_recursive_fraction_sig1 = 0.415163 [Auto,Clean] 
    0x7fdf49b364c8/V- RooRealVar::sig1frac = 0.837334 +/- 0.116852
    0x7fdf49b2e000/V- RooRealVar::bkgfrac = 0.504184 +/- 0.0359091
  0x7fdf49b35fe0/V- RooGaussian::sig2 = 1 [Auto,Dirty] 
   

Draw the frame on the canvas

In [16]:
new TCanvas("rf201_composite", "rf201_composite", 600, 600);
gPad->SetLeftMargin(0.15);
xframe->GetYaxis()->SetTitleOffset(1.4);
xframe->Draw();

Draw all canvases 

In [17]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()